
# Sujet 8 — Spectral Clustering + KDDCup 99

## Objectifs
Mettre en œuvre le **Spectral Clustering** sur le jeu de données **KDDCup99**, préparer les données,
définir des métriques pertinentes et comparer à une baseline simple.

Seed fixe : **42**


In [ ]:

# Setup & versions
import numpy as np
import pandas as pd
import random
import warnings
warnings.filterwarnings("ignore")

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

print("scikit-learn version:", sklearn.__version__)



## 1. Chargement du jeu de données (KDDCup99)
On utilise un sous-ensemble pour réduire le coût mémoire.


In [ ]:

from sklearn.datasets import fetch_kddcup99

data = fetch_kddcup99(
    subset="SA",
    percent10=True,
    as_frame=True,
    random_state=SEED
)

X = data.data
y = data.target

print("Dimensions:", X.shape)
print("Distribution des classes:")
print(y.value_counts())
X.head()



## 2. Exploration rapide


In [ ]:

print("Valeurs manquantes:", X.isna().sum().sum())
print("Types de variables:")
print(X.dtypes.value_counts())



## 3. Prétraitement
- Encodage des variables catégorielles
- Standardisation
- PCA pour réduire la dimension (nécessaire pour Spectral Clustering)


In [ ]:

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA

X_enc = X.copy()

for col in X_enc.select_dtypes(include=["object"]).columns:
    le = LabelEncoder()
    X_enc[col] = le.fit_transform(X_enc[col])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_enc)

pca = PCA(n_components=20, random_state=SEED)
X_pca = pca.fit_transform(X_scaled)

print("Shape après PCA:", X_pca.shape)



## 4. Modèle Baseline : KMeans


In [ ]:

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score

k = len(np.unique(y))

kmeans = KMeans(n_clusters=k, random_state=SEED)
labels_km = kmeans.fit_predict(X_pca)

print("KMeans Silhouette:", silhouette_score(X_pca, labels_km))
print("KMeans Davies-Bouldin:", davies_bouldin_score(X_pca, labels_km))



## 5. Spectral Clustering


In [ ]:

from sklearn.cluster import SpectralClustering

spectral = SpectralClustering(
    n_clusters=k,
    affinity="nearest_neighbors",
    n_neighbors=10,
    random_state=SEED
)

labels_sc = spectral.fit_predict(X_pca)

print("Spectral Silhouette:", silhouette_score(X_pca, labels_sc))
print("Spectral Davies-Bouldin:", davies_bouldin_score(X_pca, labels_sc))



## 6. Comparaison des modèles


In [ ]:

results = pd.DataFrame({
    "Modèle": ["KMeans", "Spectral Clustering"],
    "Silhouette": [
        silhouette_score(X_pca, labels_km),
        silhouette_score(X_pca, labels_sc)
    ],
    "Davies-Bouldin": [
        davies_bouldin_score(X_pca, labels_km),
        davies_bouldin_score(X_pca, labels_sc)
    ]
})

results



## 7. Discussion & limites
- Le clustering spectral est coûteux en mémoire
- Sensible au paramètre n_neighbors
- PCA peut entraîner une perte d'information
- Les labels réels servent uniquement à fixer k
